In [1]:
import os
import pandas as pd
from IPython.display import display

In [2]:
print(os.getcwd())

/home/nbatjargal/private/capstone_project/Majorana-Neutrino-Hunt/extracted_features_csv_files


In [3]:
ls

combine_everyone.ipynb  eunice_csv_files/  labels_train.csv
combined_csv_files/     jade_csv_files/    nomin_csv_files/
description.txt         labels_test.csv    prithvi_csv_files/


In [7]:
def process_and_merge_features(
    labels_path: str,
    eunice_path: str,
    nomin_path: str,
    prithvi_path: str,
    jade_path: str,
    output_filename: str,
    output_dir: str = "combined_csv_files",
) -> pd.DataFrame:
    """
    Loads feature CSVs from different sources, applies specific preprocessing 
    (specifically to Jade's IDs), merges them with labels, and saves the result.
    """
    
    # Load Data
    print("=" * 50)
    print("Loading Files...")
    print("=" * 50)
    
    try:
        labels = pd.read_csv(labels_path)
        e = pd.read_csv(eunice_path)
        n = pd.read_csv(nomin_path)
        p = pd.read_csv(prithvi_path)
        j = pd.read_csv(jade_path)
    except FileNotFoundError as err:
        print(f"Error loading files: {err}")
        return None

    # Fix Jade IDs (Specific Logic)
    print("Preprocessing Jade IDs...")
    g = j["id"].str.replace("id_train_", "", regex=False).astype(int)
    file_idx = (g // 65000).astype(int)
    j["id"] = g.astype(str) + "_train_" + file_idx.astype(str)

    # Preview Data    
    data_map = {
        "Labels": labels,
        "Eunice": e,
        "Nomin": n,
        "Prithvi": p,
        "Jade": j
    }

    for name, df in data_map.items():
        print(f"\n{name} — File Shape: {df.shape}")
        display(df.head())

    # Merge
    print("\n" + "=" * 50)
    print("Merging Datasets...")
    
    merged = (
        labels
        .merge(e, on="id", how="inner")
        .merge(n, on="id", how="inner")
        .merge(p, on="id", how="inner")
        .merge(j, on="id", how="inner")
    )

    print(f"Final Merged Shape: {merged.shape}")

    # Save Output
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, output_filename)
    
    print(f"Saving to {out_path}...")
    merged.to_csv(out_path, index=False, compression="gzip")
    print("Save Complete.")

    return merged

## Combine Training Files

In [8]:
paths_train = {
    "labels_path": "labels_train.csv",
    "eunice_path": "eunice_csv_files/train_csv/train_all_features.csv.gz",
    "nomin_path": "nomin_csv_files/combined_train_n.csv.gz",
    "prithvi_path": "prithvi_csv_files/train_2.csv.gz",
    "jade_path": "jade_csv_files/train_jade_features.csv"
}

merged_df_train = process_and_merge_features(
    **paths_train,
    output_filename="combined_train_with_labels.csv.gz")

merged_df_train.head()

Loading Files...
Preprocessing Jade IDs...

Labels — File Shape: (1040000, 7)


,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0
0,0_train_0,582.364295,False,True,True,True,957
1,1_train_0,250.159995,False,True,True,True,948
2,2_train_0,1212.323954,False,True,False,True,965
3,3_train_0,240.878110,False,True,True,False,927
4,4_train_0,285.124189,False,True,True,False,958



Eunice — File Shape: (1040000, 6)


,id,ED,HWP,LQ80,PPR,SC
0,0_train_0,3409.0,2299.0,-717094.898532,0.719376,0.034655
1,1_train_0,3404.0,2446.0,-331957.541919,0.729709,0.035314
2,2_train_0,3411.0,2262.0,-425532.152706,0.715390,0.034915
3,3_train_0,3408.0,2833.0,-306980.459766,0.769375,0.034752
4,4_train_0,3406.0,2397.0,-362746.925366,0.728165,0.035132



Nomin — File Shape: (1040000, 9)


...

,id,tdrift99,tfr,peak_count,gbn,bpr
0,0_train_0,85.0,0.142357,3,1.198436,0.059642
1,1_train_0,87.0,0.151160,3,1.381123,0.061975
2,2_train_0,95.0,0.142606,3,1.361857,0.050813
3,3_train_0,116.0,0.133192,2,1.165654,0.062954
4,4_train_0,94.0,0.145504,7,1.390918,0.055013



Jade — File Shape: (1040000, 5)


,id,AvsE,GradAreaRatio,GradWidthMain,HFER
0,0_train_0,0.519805,1.0,76.0,0.034772
1,1_train_0,0.531775,1.0,46.0,0.037621
2,2_train_0,0.343676,1.0,90.0,0.035552
3,3_train_0,0.483144,1.0,56.0,0.035093
4,4_train_0,0.377738,1.0,53.0,0.035971



Merging Datasets...
Final Merged Shape: (1040000, 29)
Saving to combined_csv_files/combined_train_with_labels.csv.gz...
Save Complete.


,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,late_over_early,tdrift99,tfr,peak_count,gbn,bpr,AvsE,GradAreaRatio,GradWidthMain,HFER
0,0_train_0,582.364295,False,True,True,True,957,3409.0,2299.0,-717094.898532,...,0.987910,85.0,0.142357,3,1.198436,0.059642,0.519805,1.0,76.0,0.034772
1,1_train_0,250.159995,False,True,True,True,948,3404.0,2446.0,-331957.541919,...,0.988301,87.0,0.151160,3,1.381123,0.061975,0.531775,1.0,46.0,0.037621
2,2_train_0,1212.323954,False,True,False,True,965,3411.0,2262.0,-425532.152706,...,0.987491,95.0,0.142606,3,1.361857,0.050813,0.343676,1.0,90.0,0.035552
3,3_train_0,240.878110,False,True,True,False,927,3408.0,2833.0,-306980.459766,...,0.988450,116.0,0.133192,2,1.165654,0.062954,0.483144,1.0,56.0,0.035093
4,4_train_0,285.124189,False,True,True,False,958,3406.0,2397.0,-362746.925366,...,0.988541,94.0,0.145504,7,1.390918,0.055013,0.377738,1.0,53.0,0.035971


In [9]:
df_train = pd.read_csv('combined_csv_files/combined_train_with_labels.csv.gz')
df_train

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,late_over_early,tdrift99,tfr,peak_count,gbn,bpr,AvsE,GradAreaRatio,GradWidthMain,HFER
0,0_train_0,582.364295,False,True,True,True,957,3409.0,2299.0,-717094.898532,...,0.987910,85.0,0.142357,3,1.198436,0.059642,0.519805,1.000000,76.0,0.034772
1,1_train_0,250.159995,False,True,True,True,948,3404.0,2446.0,-331957.541919,...,0.988301,87.0,0.151160,3,1.381123,0.061975,0.531775,1.000000,46.0,0.037621
2,2_train_0,1212.323954,False,True,False,True,965,3411.0,2262.0,-425532.152706,...,0.987491,95.0,0.142606,3,1.361857,0.050813,0.343676,1.000000,90.0,0.035552
3,3_train_0,240.878110,False,True,True,False,927,3408.0,2833.0,-306980.459766,...,0.988450,116.0,0.133192,2,1.165654,0.062954,0.483144,1.000000,56.0,0.035093
4,4_train_0,285.124189,False,True,True,False,958,3406.0,2397.0,-362746.925366,...,0.988541,94.0,0.145504,7,1.390918,0.055013,0.377738,1.000000,53.0,0.035971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039995,1039995_train_15,210.834626,True,True,True,True,949,3403.0,2533.0,-278607.352936,...,0.985736,90.0,0.151747,6,1.008092,0.062825,0.577970,58.524510,57.0,0.034958
1039996,1039996_train_15,380.170340,True,True,True,True,949,3404.0,2426.0,-477777.966558,...,0.987814,79.0,0.150305,3,1.245492,0.062706,0.643747,80.265957,45.0,0.033676
1039997,1039997_train_15,370.109563,False,True,True,True,947,3409.0,2417.0,-466589.983952,...,0.987833,102.0,0.142950,2,1.149456,0.062153,0.480104,1.000000,59.0,0.035684
1039998,1039998_train_15,98.258524,True,True,True,False,921,3400.0,3799.0,-143212.214717,...,0.989948,92.0,0.153135,5,1.192639,0.065264,0.483468,1.000000,97.0,0.034234


In [10]:
df_train.describe()

,energy_label,tp0,ED,HWP,LQ80,PPR,SC,current_skewness,spectral_centroid_power,tail_charge_diff,...,late_over_early,tdrift99,tfr,peak_count,gbn,bpr,AvsE,GradAreaRatio,GradWidthMain,HFER
count,1.040000e+06,1.040000e+06,1.040000e+06,1.039995e+06,1.040000e+06,1.039995e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,...,1.040000e+06,1.039995e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06,1.040000e+06
mean,6.381633e+02,9.516434e+02,3.406036e+03,2.423961e+03,1.583331e+124,7.254552e-01,3.572992e-02,1.906792e+00,1.145299e+02,-5.468692e-01,...,9.893650e-01,1.131437e+02,1.495163e-01,5.104236e+00,1.777303e+00,6.049704e-02,4.964875e-01,4.091337e+11,6.792718e+01,3.643588e-02
std,6.704271e+02,1.895332e+01,7.735526e+00,3.925200e+02,1.614687e+127,2.832694e-02,6.330801e-03,5.312883e-01,1.069477e+02,1.561871e+00,...,1.121635e-01,7.287391e+01,3.987192e-02,7.352031e+00,1.174284e+00,2.152323e-02,9.173632e-02,1.401446e+12,2.442059e+01,8.420477e-03
min,0.000000e+00,7.100000e+01,3.074000e+03,2.000000e+00,-5.949322e+06,-7.703542e+00,2.676319e-02,-1.809541e+00,8.986718e+01,-8.692308e+00,...,1.302615e-01,9.000000e+00,1.968922e-02,1.000000e+00,6.992497e-01,3.636530e-02,3.150645e-02,1.000000e+00,5.000000e+00,2.650746e-02
25%,2.383612e+02,9.420000e+02,3.404000e+03,2.191000e+03,-7.774117e+05,7.088948e-01,3.452545e-02,1.584701e+00,1.067439e+02,-6.236000e-01,...,9.873562e-01,8.600000e+01,1.408390e-01,1.000000e+00,1.116426e+00,5.533778e-02,4.353313e-01,1.000000e+00,5.100000e+01,3.454584e-02
50%,3.411866e+02,9.540000e+02,3.407000e+03,2.325000e+03,-4.279560e+05,7.211736e-01,3.480813e-02,1.939160e+00,1.086075e+02,-5.968707e-01,...,9.879051e-01,1.020000e+02,1.457415e-01,2.000000e+00,1.321930e+00,6.133205e-02,5.268103e-01,1.000000e+00,6.500000e+01,3.505329e-02
75%,6.300128e+02,9.640000e+02,3.410000e+03,2.510000e+03,-3.063921e+05,7.361495e-01,3.524030e-02,2.253204e+00,1.103433e+02,-5.640222e-01,...,9.885530e-01,1.210000e+02,1.509235e-01,5.000000e+00,1.908089e+00,6.423424e-02,5.653386e-01,6.588437e+01,8.200000e+01,3.580735e-02
max,4.998508e+03,1.352000e+03,3.790000e+03,3.799000e+03,1.646664e+130,9.768917e-01,2.028693e-01,5.229496e+00,1.637442e+04,3.112500e+02,...,4.640625e+01,2.919000e+03,3.990051e+00,5.500000e+01,3.667323e+01,2.569411e+00,9.706884e-01,7.968905e+12,3.910000e+02,2.743541e-01


## Combine Test Files

In [ ]:
paths_test = {
    "labels_path": "labels_test.csv",
    "eunice_path": "eunice_csv_files/test_csv/...",
    "nomin_path": "nomin_csv_files/combined_test_n.csv.gz",
    "prithvi_path": "prithvi_csv_files/test_2.csv.gz",
    "jade_path": "jade_csv_files/test_jade_features.csv"
}

merged_df_test = process_and_merge_features(
    **paths_test,
    output_filename="combined_test_with_labels.csv.gz")

merged_df_test.head()

In [ ]:
df_test = pd.read_csv('combined_csv_files/combined_test_with_labels.csv.gz')
df_test

In [ ]:
df_test.describe()